In [ ]:
#wenn Notebook auf https://lab.quantum-computing.ibm.com ist diese Zelle automatisch bereits geladen
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
#wenn Notebook extern QISKIT noch mit !pip install qiskit installieren
import qiskit
from numpy import pi

#wenn extern: Account manuell mit Token laden, wenn mit Quantencomputer kommuniziert werden soll
from qiskit import IBMQ
provider = IBMQ.enable_account('HIER Token von Q-Account')

## QFT mit Qiskit

In [ ]:
# (1) Initiieren des Quantenschaltkreises mit 3 Qubits
QFTcircuit = qiskit.QuantumCircuit(3, 3)

# (2) Zustand angeben, welcher in Fourier-Basis übertragen werden soll
init_state = [1, 1, 0]
for qubit, state in enumerate(init_state):
    if state == 1:
        QFTcircuit.x(qubit)

# (3) jeden Qubit mit H-Gatter in die Fourier-Basis überführen
QFTcircuit.h(0)

# (4) Qubits über CU1-Gatter miteinander als gemeinsames QFT verknüpfen
QFTcircuit.cp(pi/2, 1, 0)      #CU1 = 𝜋/2
QFTcircuit.cp(pi/4, 2, 0)      #CU1 = 𝜋/4 -> langsam

#rekursiv für q1 
QFTcircuit.h(1)                 #H = 𝜋
QFTcircuit.cp(pi/2, 2, 1)      #CU1 = 𝜋/2 -> mittel

#rekursiv für q2
QFTcircuit.h(2)                 #H = 𝜋 -> schneller Qubit

#QFTcircuit.measure([0,1,2], [0,1,2])
QFTcircuit.draw(output='mpl')

### Simulation des Circuits auf der Bloch-Sphäre

In [ ]:
#Simulieren auf der Bloch-Sphäre
backend = qiskit.BasicAer.get_backend('statevector_simulator')
job = qiskit.execute(QFTcircuit, backend).result()

#Darstellung der Simulation als Bild
qiskit.visualization.plot_bloch_multivector(job.get_statevector(QFTcircuit))

### Messungen auf echtem Quantencomputer

In [ ]:
#Messungen mit echtem Quantencomputer über die Cloud
#Auflisten aller verfügbaren Backends (über https://lab.quantum-computing.ibm.com mehr verfügbar)
provider.backends()

In [ ]:
#Quantencomputer als Backend einstellen
#gerne mit anderen probieren -> Achtung: Armonk zu wenig Qubits
q_backend = provider.get_backend('ibmq_quito')

#Wahrscheinlichkeiten werden gemessen -> mehrere Messungen/Shots
q_job = qiskit.execute(QFTcircuit, q_backend, shots=1000).result()
counts = q_job.get_counts(QFTcircuit)
qiskit.visualization.plot_histogram(counts, title="Q-Computer")